### Prompting Engineering

In [1]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
#환경설정
import os
from dotenv import load_dotenv

load_dotenv() # 현재 경로의 .env 파일을 읽어 시스템 환경변수로 등록

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

### Chat Completion

In [5]:
from openai import OpenAI

# clinet-server
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role":"system", 
            "content":[
                {
                    "type":"text",
                    "text":"당신은 아주 유쾌한 해적 선장입니다."
                }
            ]
        },
        {
            "role":"user", 
            "content":[
                {
                    "type":"text",
                    "text":"추운날 점심에는 뭘 먹으면 좋을까요?"
                }
            ]
        }
    ],
    response_format={ "type": "text" },
    temperature=1.3,    # 대답 창의성(기본값 1) 0~2
    max_tokens= 2048,   # 응답 최대 토큰수
    top_p=1             # 사용할 상위누적확률
)

print(response.choices[0].message.content)

아, 차가운 바람이 불어오는 날에는 뱃속을 따뜻하게 하는 한 끼가 딱이지! 이런 날에는 매콤한 김치찌개나 얼큰한 고추장을 풀어 넣은 누룽지탕 같은 음식이 잘 어울려. 아니면 따끈한 설렁탕이나 육개장 한 그릇도 최고지. 뜨거운 국물과 함께라면 바닷바람도 두렵지 않을걸! 맛있게 먹고 한층 더 따스한 기분으로 하루를 보내기 바란다네!


### 패턴1: 페르소나 & Few-Shot(기사 제목 교정)

In [8]:
from openai import OpenAI

def correct_title(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 신문사의 베테랑 편집장입니다. 기사들이 송고한 제목을 교정해주세요.

    ### 지시사항 ###
    - 기사의 제목이 명확하고 주제와 잘 맞도록 수정해주세요
    - 비속어, 은어등은 제거하고 의미가 유지되도록 제목을 교정해주세요.
    - 간결하고 임팩트 있는 표현을 사용해주세요.

    ### 출력 예시 ###
    - 원래 제목 : [기사의 원래 제목]
    - 교정 제목 :
        [교정된 제목1]
        [교정된 제목2]

    ## 예시 ###
    - 원래 제목 : "어제 서울에서 큰불이 나서 수백명이 대피했다"
    - 교정 제목 : 
        "서울 대형화재, 수백명 대피"
        "수백명이 큰불로 도망!"
    """

    user_message=f"""
    다음 제목을 교정해주세요.
    제목 : {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )

    return response.choices[0].message.content

print(correct_title('이은지 FM 개꿀 라디오 방송에 주목해주세요'))
print(correct_title('졸라 빡쎈 작업으로 끼니를 거르는 이 시대 노동자의 애환'))


    - 원래 제목 : 이은지 FM 개꿀 라디오 방송에 주목해주세요
    - 교정 제목 : 
        "이은지 FM 라디오 방송, 주목해야 할 이유"
        "이은지 FM 라디오, 청취자들의 관심 집중"

    - 원래 제목 : 졸라 빡쎈 작업으로 끼니를 거르는 이 시대 노동자의 애환
    - 교정 제목 :
        "고된 노동으로 끼니를 거르는 현대 노동자의 현실"
        "현대 노동자의 고된 작업과 끼니 문제"


### 구조화된 출력 (Structured Ouptput) - 영단어장 생성

In [ ]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def extract_eng_words(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 영어팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

    # 처리단계
    1. 주어진 가사에서 자주 사용되는 영어단더 5개를 랜덤으로 추출해주세요.
    2. 각 단어의 의미를 한글로 알려주세요.
    3. 단어별로 유사한 단어도 함께 소개해주세요.

    # 출력형식
    출력형식은 json 형식입니다.
    - 최상위 키는 "json_list" 여야 합니다.
    -"json_list"의 값은 단어별 json 객체들이 담긴 리스트여야 합니다.

    # 출력형식
    {
        "json_list" : [
            {
                "단어" : "yesterday",
                "의미" : "어제"
                "유사어" : [
                    {
                        "유사단어" : "today"    
                    }, 
                    {
                        "유사단어" : "tomorrow"
                    }
                ]
            }
        ]
    }
    """

    user_message=f"""
    노래가사: {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "json_object" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return json.loads(response.choices[0].message.content) # 응답 json문자열을 파이썬 객체로 변환 

lyrics="Yesterday all my troubles seemed so far away Now it looks as though they're here to stay Oh, I believe in yesterday Suddenly I'm not half the man I used to be There's a shadow hanging over me Oh, yesterday came suddenly Why she had to go I don't know, she wouldn't say I said something wrong now I long for yesterday Yesterday love was such an easy game to play Now I need a place to hide away Oh, I believe in yesterday. Why she had to go I don't know, She wouldn't say I said something wrong Now I long for yesterday Yesterday love was such an easy game to play Now I need a place to hide away Oh, I believe in yesterday"

result = extract_eng_words(lyrics, temperature=1)
words = result['json_list']

# 파이썬 객체로 변환되었기 때문에 for 반복문을 통해 각 요소를 순회하고 딕셔너리의 키를 이용해 원하는 데이터에 직접 접근 가능

for word_dict in words:
    print(f"단어 : {word_dict["단어"]}")
    print(f"의미 : {word_dict["의미"]}")



단어 : yesterday
의미 : 어제
단어 : troubles
의미 : 문제들
단어 : believe
의미 : 믿다
단어 : suddenly
의미 : 갑자기
단어 : shadow
의미 : 그림자


### 생각의 사슬(Chain-of-Thought) (냉털 마스터)- ReAct
- Reason + Act 기법으로 현재 상황에 대한 통찰이후, 다음 행동에 대한 작성을 유도하는 기법

In [17]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def my_refregator(query, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    너는 냉장고에 있는 재료들을 활용하여 창의적이고 실용적인 저녁식사 아이디어를 제안하는 요리 전문가이다.
    네 역할은 사용자로부터 제공받은 재료목록을 분석하고, 이를 활용할 수 있는 요리 아이디어를 구상해서,
    조리방법을 단계별로 상세히 설명하는 것이다.

    # 출력예시
    1. 상황분석
    - 현재 가진 재료는 [사용자가 입력한 재료] 입니다.
    - 주재료인 닭고기와 다양한 채소들이 있습니다.
    - 이 재료들은 스튜, 볶음, 찜요리에 적합합니다.

    2. 행동계획
    - 가장 쉽게 만들수 있는 요리로 닭볶음탕을 제안합니다.
    - 각 요리에 필요한 재료와 조리도구를 확인합니다.
    - 부족한 재료가 있다면, 대체가능한 옵션을 제시합니다.
    - 조리 과정을 단계별로 상세히 설명합니다.
    - 맛을 더 향상시킬수 있는 팁과 주의사항 또한 제공합니다. 

    3. 실행
    - 여기에 상세레시피를 단계별로 작성합니다.
    """

    user_message=f"""
   사용자의 냉장고에는 현재 {query}가 있습니다.
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return response.choices[0].message.content # 응답 json문자열을 파이썬 객체로 변환 

print(my_refregator("감자, 소고기, 된장, 표고버섯", temperature=1.2))

1. 상황분석
   - 현재 가진 재료는 감자, 소고기, 된장, 표고버섯입니다.
   - 감자와 소고기가 주재료로, 된장으로 맛을 낼 수 있습니다.
   - 이 재료들은 국물 요리나 조림 요리에 적합합니다.

2. 행동계획
   - 가장 쉽게 만들 수 있는 요리로 '소고기 감자 된장찌개'를 제안합니다.
   - 필요한 재료: 감자, 소고기, 된장, 표고버섯, 추가적으로 필요할 수 있는 양념(간장, 다진 마늘, 후추 등)과 물.
   - 해당 요리에 사용하는 다른 야채(양파, 대파 등)가 필요하지만, 대체 가능하다면 스킵 가능합니다.
   - 조리 과정을 단계별로 설명합니다.
   - 맛을 더 향상시킬 수 있는 팁과 조리 시 주의사항을 포함합니다.

3. 실행

   - 단계별 레시피

     1. 손질 작업
        - 감자: 깨끗이 씻고 적당한 크기로 깍둑썰기 합니다.
        - 소고기: 먹기 좋은 크기로 썰어 준비합니다.
        - 표고버섯: 세척 후 얇게 슬라이스 합니다.
 
     2. 기본 육수 준비
        - 된장을 물에 풀어 된장 기반 육수를 만듭니다. (가급적이면 약 4컵 정도 준비)

     3. 찌개 끓이기
        - 냄비에 약간의 기름을 두르고, 다진 마늘을 넣어 향을 냅니다.
        - 여기에 소고기를 넣고 겉면이 살짝 익을 때까지 볶아줍니다. 간장 약간을 추가하여 밑 맛을 더합니다.
        - 미리 준비한 된장 육수를 붓고, 끓어오르면 감자와 표고버섯을 넣어줍니다.
        - 재료들이 모두 익을 때까지 중불에서 끓입니다. 

     4. 간 조절과 마무리 
        - 간이 필요하면 소금, 간장 등을 추가로 사용해 자유롭게 조절합니다.
        - 후추를 뿌려 맛을 더한 뒤, 한소끔 더 끓이고 불을 끕니다.

     5. 맛 향상 팁
        - 대파나 고춧가루를 조금 넣어 더 깊은 맛을 추구해볼 수 있습니다.
        - 마지막에 참기름을 살짝 더하여

### 면접 질문 생성

In [18]:
from openai import OpenAI
import json # API로부터 받은 JSON형식의 텍스트 응답을 파이썬 객체로 변환하는데 사용

def job_interview(job_posting, temperature=0.3):

    # clinet-server
    client = OpenAI(api_key=OPENAI_API_KEY)

    system_instruction="""
    당신은 풀스택과 AI 개발 분야의 전문가이면서, 해당 분야의 면접 전문가입니다.
    사용자가 제공한 구인공고에 근거해서 핵심적인 질문들로 지원자를 평가할 수 있어야 합니다.
    """

    user_message=f"""
    아래 채용공고의 직무에 대해 예상면접 질문과 모범답안을 작성해주세요.

    -- 가이드 --
    하드스킬과 소프트스킬 2개의 섹션으로 나눠 작성해 주세요.
    각 섹션별로 2개의 질문과 답변을 준비해주세요.

    -- 출력형식 --
    # 1. 하드스킬
    질문1)
    답변1)

    질문2)
    답변2)

    # 2. 소프트스킬
    질문1)
    답변1)

    질문2)
    답변2)

    -- 채용공고 --
    {job_posting}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_instruction
                    }
                ]
            },
            {
                "role":"user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_message
                    }
                ]
            }
        ],
        response_format={ "type": "text" },
        temperature=temperature,    # 대답 창의성(기본값 1) 0~2
        max_tokens= 2048,   # 응답 최대 토큰수
        top_p=1             # 사용할 상위누적확률
    )


    return response.choices[0].message.content # 응답 json문자열을 파이썬 객체로 변환 

job_posting = """
담당업무
- 웹 서비스 운영 및 개발

- 기획 협의 및 화면 구현

- GitLab 기반 소스 관리


자격요건
- 학사 이상

- 경력 : 5년 이내 

- 전공: 컴퓨터공학 및 전산 관련 전공하신 분

- 웹 개발 경험 및 협업 툴 사용 경험이 있으신 분

- 기본적인 코드 리뷰 대응 능력을 보유하신 분

- React, Node.js, Asp.net 활용 가능 하신 분


우대사항
- React, Svelte 프로젝트 경험이 있으신 분

- UI/UX에 대한 이해도가 있으신 분

- 웹 접근성 고려 개발 경험 및 Git 기반 현업 경험이 있으신 분


공통 우대사항
- 주식시장과 기업공시에 대한 기본 지식을 갖추신분

- 빠른 면접진행 및 출근가능자 우대


기타사항
- 긍정 마인드, 책임감, 솔선수범
"""

print(job_interview(job_posting, temperature=1))

# 1. 하드스킬

질문1)  
React와 Node.js를 사용하여 웹 애플리케이션을 개발한 경험이 어떻게 되시나요? 주요 프로젝트 하나를 설명해주시고, 이 과정에서 직면했던 큰 문제와 해결 방법에 대해 말씀해주세요.

답변1)  
저는 React와 Node.js를 사용하여 한 대형 전자 상거래 플랫폼을 개발한 경험이 있습니다. 이 프로젝트는 수천 명의 동시 사용자를 지원해야 했기 때문에 서버 성능 최적화가 큰 문제였습니다. 이를 해결하기 위해 Node.js의 비동기 처리 방식을 적극 활용하여, 데이터베이스 요청을 효율적으로 관리하고, 페이지 로딩 속도를 개선했습니다. 또한, 프론트엔드 최적화를 위해 코드 스플리팅과 캐싱 전략을 적극 도입했습니다.

질문2)  
GitLab을 사용한 경험이 있다고 하셨는데, GitLab을 사용하여 팀 협업과 코드 관리를 어떻게 효율적으로 수행했는지 설명해 주시겠어요?

답변2)  
GitLab을 사용하여 여러 프로젝트에서 CI/CD 파이프라인을 설정하였습니다. 메인 브랜치로 통합되기 전, 모든 코드 변경이 자동으로 테스트되도록 구성하여, 잠재적인 버그를 조기에 발견할 수 있었습니다. 또한, 코드 리뷰 프로세스를 적극 활용하여 팀원들이 각자의 코드를 검토하게 했습니다. 이를 통해 코드 품질을 높이는 동시에, 팀원들 간 지식 공유도 자연스럽게 이루어졌습니다.

# 2. 소프트스킬

질문1)  
팀과 함께 협업할 때 가장 중요한 덕목은 무엇이라고 생각하시나요? 예전의 경험을 바탕으로 설명해 주세요.

답변1)  
저는 팀과의 협업에서 가장 중요한 덕목은 '커뮤니케이션과 투명성'이라고 생각합니다. 이전에 진행했던 프로젝트에서는 처음 기획 단계에서 의사소통이 부족해 이해 차이가 발생했고, 이로 인해 개발 일정이 지연된 적이 있었습니다. 이 경험을 통해 이후에는 팀원 간 주기적인 회의를 통해 진행 상황과 문제점을 공유하고, 모든 결정 사항은 문서화하여 투명성을 유지했습니다. 이는 프로젝트의 원활한 진행에 크게 기여했습니다.

질문2) 